In [1]:
import pandas as pd
import sys
from indicnlp import common

In [2]:
hindi_tweets = pd.read_csv("hindilabelled.csv")
#hindi_tweets.head()

In [3]:
hindi_tweets['CAA_stance'].value_counts()

Anti_CAA    17575
Pro_CAA     16060
Neutral      2470
Name: CAA_stance, dtype: int64

In [4]:
#from indic_transliteration import sanscript
#from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate


#print(transliterate(data, sanscript.ITRANS, sanscript.DEVANAGARI))


In [5]:
#hindi_tweets["text"] = hindi_tweets["text"].map(lambda x: transliterate(x, sanscript.ITRANS, sanscript.DEVANAGARI))

In [6]:
hindi_twee = hindi_tweets[hindi_tweets["CAA_stance"].notnull()]

import numpy as np
from random import sample

#np.random.shuffle(hindi_twee.values)

hindi_tweet = hindi_twee[:int((len(hindi_twee)+1)*.30)]

In [7]:
hindi_tweet['CAA_stance'].value_counts()

Pro_CAA     5721
Anti_CAA    2912
Neutral     2198
Name: CAA_stance, dtype: int64

In [8]:
from io import StringIO
col = ['CAA_stance', 'text']
hindi_tweet1 = hindi_tweet[col]
hindi_tweet1 = hindi_tweet1[pd.notnull(hindi_tweet1['text'])]
hindi_tweet1.columns = ['CAA_stance', 'text']
hindi_tweet1['stance_id'] = hindi_tweet1['CAA_stance'].factorize()[0]
stance_id_df = hindi_tweet1[['CAA_stance', 'stance_id']].drop_duplicates().sort_values('stance_id')
stance_to_id = dict(stance_id_df.values)
id_to_stance = dict(stance_id_df[['stance_id', 'CAA_stance']].values)
#hindi_tweet1.head(10)

In [9]:
import nltk

In [10]:
hi = pd.read_csv("final_stopwords.txt", names=['words'])
hieng = pd.read_csv("hinglish_stopwords.txt", names = ['words'])

In [11]:
hi = hi['words'].to_list()
hieng = hieng['words'].to_list()

In [12]:
from nltk.corpus import stopwords
final_stopwords_list = hi

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf = True, min_df = 5, norm ='l2', encoding='ISCII', ngram_range=(1, 2), stop_words = final_stopwords_list)
features = tfidf.fit_transform(hindi_tweet1.text).toarray()
labels = hindi_tweet1.stance_id
features.shape

/Users/shreyaagarwal/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['अथव', 'अध', 'अन', 'अपन', 'अभ', 'आग', 'आद', 'आपक', 'इत', 'इनक', 'इसक', 'इसम', 'इसल', 'उनक', 'उसक', 'एव', 'ऐस', 'कभ', 'करत', 'करन', 'कह', 'कहत', 'गय', 'जबक', 'जर', 'जह', 'झक', 'तथ', 'तन', 'तर', 'दर', 'धर', 'नस', 'पड', 'पहल', 'बड', 'बन', 'बह', 'यत', 'यद', 'यम', 'रख', 'रत', 'रह', 'रहत', 'लक', 'वय', 'वर', 'वग़', 'सक', 'सकत', 'सभ', 'सम', 'सर', 'सस', 'हमन', 'हर'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(10831, 5274)

In [14]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize    
from sklearn.model_selection import cross_val_score

model = LinearSVC()
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, hindi_tweet1.index, test_size=0.33, random_state=0)

clf = model.fit(X_train, y_train)
scores = cross_val_score(clf, X_test, y_test, cv = 5)

In [15]:
print(scores)

[0.83776224 0.81958042 0.83356643 0.80839161 0.85314685]


In [16]:
import pickle
filename = 'finalized_modelhindi.sav'
pickle.dump(model, open(filename, 'wb')) 

In [17]:

loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, y_test)



In [18]:
from sklearn.metrics import classification_report
 
print (classification_report(y_test, model.predict(X_test), target_names=['Pro_CAA', 'Anti_CAA', 'Neutral']))



              precision    recall  f1-score   support

     Pro_CAA       0.87      0.89      0.88      1890
    Anti_CAA       0.78      0.74      0.76       971
     Neutral       0.91      0.92      0.91       714

    accuracy                           0.86      3575
   macro avg       0.85      0.85      0.85      3575
weighted avg       0.85      0.86      0.86      3575



In [94]:
keys = list(hindi_tweet.columns.values)
i1 = hindi_tweets.set_index(keys).index
i2 = hindi_twee.set_index(keys).index
tobelabelled = hindi_tweets[~i1.isin(i2)]
tobelabelled.shape



(83926, 10)

In [95]:
x_new = tfidf.transform(tobelabelled.text)
x_new = x_new

In [96]:
y_pred = loaded_model.predict(x_new)

In [97]:
tobelabelled["CAA_stance"] = y_pred

/Users/shreyaagarwal/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [98]:
frames = [tobelabelled, hindi_twee]
hi_tweets = pd.concat(frames)

In [102]:
file_name = "/Users/shreyaagarwal/Google Drive/Dissertation/Data/hindilabelled_1.csv"

In [103]:
hi_tweets.to_csv(file_name, sep='\t', encoding='utf-8')

In [101]:
hi_tweets["CAA_stance"].value_counts()

0           35956
2           26374
1           21596
Anti_CAA    17575
Pro_CAA     16060
Neutral      2470
Name: CAA_stance, dtype: int64